In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import torch
from my_dqn import ReplayBuffer
import viz_helper

c:\Users\Marius\anaconda3\envs\general\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
import datetime

In [7]:
time_start = datetime.datetime.now() 

In [8]:
datetime.datetime.now()  - time_start 

datetime.timedelta(seconds=7, microseconds=488206)

In [3]:
game = "breakout"

load_file_path = os.path.join("breakout", game + "_checkpoint")

training_stats, validation_stats = viz_helper.read_training_logs(load_file_path)

In [14]:
fig, axs = plt.subplots(nrows=2, ncols=1)

viz_helper.plot_episodic_logs(
    fig,
    axs,
    training_stats["avg_episode_rewards"],
    training_stats["avg_episode_nr_frames"],
    training_stats["log_frame_stamp"],
    "Training stats",
)
plt.show()

In [15]:
fig, axs = plt.subplots(nrows=2, ncols=1)

viz_helper.plot_episodic_logs(
    fig,
    axs,
    validation_stats["val_avg_episode_rewards"],
    validation_stats["val_avg_episode_nr_frames"],
    validation_stats["val_log_frame_stamp"],
    "Validation stats",
)
plt.show()

ValueError: All arrays must be of the same length

In [17]:
for key in validation_stats:
    print(key, len(validation_stats[key]))

val_avg_episode_rewards 32
val_avg_episode_nr_frames 63
val_log_frame_stamp 63


In [23]:
validation_stats["val_avg_episode_rewards"]

[2.9580367178718623,
 2.9488041853512708,
 5.995014245014245,
 6.227575982209044,
 9.310112359550562,
 9.568131868131868,
 12.704384724186705,
 12.700276243093922,
 13.232995658465992,
 14.098918083462133,
 13.236574746008708,
 15.050324675324676,
 15.261562998405104,
 17.56954887218045,
 17.339622641509433,
 18.706751054852322,
 18.026315789473685,
 16.346296296296295,
 16.804744525547445,
 18.284584980237153,
 20.17452830188679,
 18.634743875278396,
 19.765281173594133,
 18.933333333333334,
 20.113702623906704,
 19.293827160493827,
 21.132231404958677,
 24.04597701149425,
 24.070121951219512,
 28.71232876712329,
 27.421511627906977,
 30.393939393939394]

In [3]:
from minatar import Environment

game = "breakout"
env = Environment(game)

my_agent = my_dqn.AgentDQN(env, "file_name", False, "")

In [4]:
my_agent.train(10, 10000)

INFO:root:Avg return: 0.0 | Time per frame: 0.00021054479810926648


In [5]:
current_state = my_dqn.get_state(my_agent.env.state())

current_state.size()

torch.Size([1, 4, 10, 10])

In [6]:
import torch

In [8]:
sample = my_agent.replay_buffer.sample(my_agent.batch_size)

state, action, reward, next_state, terminated = sample

state = torch.from_numpy(state)
next_state = torch.from_numpy(next_state)
action = torch.LongTensor(action).unsqueeze(1)
reward = torch.FloatTensor(reward).unsqueeze(1)
terminated = torch.FloatTensor(terminated).unsqueeze(1)

In [9]:
state.size()

torch.Size([32, 4, 10, 10])

In [10]:
action.size()

torch.Size([32, 1])

In [17]:
q_value = my_agent.policy_model(state).gather(1, action)

next_q_values = my_agent.target_model(next_state).detach()
next_q_value = next_q_values.max(1)[0].unsqueeze(1)

expected_q_value = reward + my_agent.gamma * next_q_value * (1 - terminated)
# expected_q_value = torch.unsqueeze(expected_q_value, 1)

loss = (q_value - expected_q_value).pow(2).mean()

In [18]:
loss

tensor(0.0969, grad_fn=<MeanBackward0>)

In [29]:
q_value = my_agent.policy_model(state)
q_value

tensor([[-0.0438, -0.0587, -0.0161,  0.0629, -0.0603,  0.0530],
        [-0.0463, -0.0590, -0.0196,  0.0600, -0.0618,  0.0527],
        [-0.0438, -0.0587, -0.0161,  0.0629, -0.0603,  0.0530],
        [-0.0438, -0.0587, -0.0161,  0.0629, -0.0603,  0.0530],
        [-0.0438, -0.0587, -0.0161,  0.0629, -0.0603,  0.0530],
        [-0.0438, -0.0587, -0.0161,  0.0629, -0.0603,  0.0530],
        [-0.0438, -0.0587, -0.0161,  0.0629, -0.0603,  0.0530],
        [-0.0438, -0.0587, -0.0161,  0.0629, -0.0603,  0.0530],
        [-0.0438, -0.0587, -0.0161,  0.0629, -0.0603,  0.0530],
        [-0.0438, -0.0587, -0.0161,  0.0629, -0.0603,  0.0530],
        [-0.0438, -0.0587, -0.0161,  0.0629, -0.0603,  0.0530],
        [-0.0438, -0.0587, -0.0161,  0.0629, -0.0603,  0.0530],
        [-0.0438, -0.0587, -0.0161,  0.0629, -0.0603,  0.0530],
        [-0.0438, -0.0587, -0.0161,  0.0629, -0.0603,  0.0530],
        [-0.0438, -0.0587, -0.0161,  0.0629, -0.0603,  0.0530],
        [-0.0438, -0.0587, -0.0161,  0.0

In [44]:
q_value.size()

torch.Size([32, 1])

In [45]:
action.unsqueeze(0).size()

torch.Size([1, 32])

In [71]:
q_value = my_agent.policy_model(state).gather(1, action.unsqueeze(1))

In [72]:
q_value.size()

torch.Size([32, 1])

In [73]:
next_q_values = my_agent.target_model(next_state).detach()
next_q_value = next_q_values.max(1)[0].unsqueeze(1)

In [74]:
next_q_value.size()

torch.Size([32, 1])

In [83]:
terminated.size()

torch.Size([32])

In [81]:
reward.unsqueeze(1).size()

torch.Size([32, 1])

In [84]:
expected_q_value = reward.unsqueeze(1) + my_agent.gamma * next_q_value * (1 - terminated.unsqueeze(1))
# expected_q_value = torch.unsqueeze(expected_q_value, 1)
expected_q_value

tensor([[0.0429],
        [0.0429],
        [0.0429],
        [0.0429],
        [0.0429],
        [0.0429],
        [0.0429],
        [0.0429],
        [0.0429],
        [0.0429],
        [0.0429],
        [0.0429],
        [0.0429],
        [0.0429],
        [0.0429],
        [0.0429],
        [0.0429],
        [0.0429],
        [0.0429],
        [0.0429],
        [0.0429],
        [0.0429],
        [0.0429],
        [0.0429],
        [0.0429],
        [0.0429],
        [0.0429],
        [0.0429],
        [0.0429],
        [0.0429],
        [0.0429],
        [0.0429]])

In [85]:
expected_q_value.size()

torch.Size([32, 1])

In [86]:
loss = (q_value - expected_q_value).pow(2).mean()

In [87]:
loss

tensor(0.0049, grad_fn=<MeanBackward0>)

In [95]:
state, action, reward, next_state, terminated = sample



In [122]:
state, action, reward, next_state, terminated = sample

state = torch.from_numpy(state)
next_state = torch.from_numpy(next_state)
action = torch.LongTensor(action).unsqueeze(1)
reward = torch.FloatTensor(reward).unsqueeze(1)
terminated = torch.FloatTensor(terminated).unsqueeze(1)

In [123]:
q_value = my_agent.policy_model(state).gather(1, action)
q_value.size()

torch.Size([32, 1])

In [126]:
next_q_values = my_agent.target_model(next_state).detach()
next_q_values = next_q_values.max(1)[0].unsqueeze(1)
expected_q_value = reward + my_agent.gamma * next_q_values * (1 - terminated)
expected_q_value.size()

torch.Size([32, 1])

In [130]:
loss = f.smooth_l1_loss(expected_q_value, q_value)
loss


tensor(0.0024, grad_fn=<SmoothL1LossBackward0>)

In [116]:
Q_s_prime_a_prime = torch.zeros(len(state), 1)
if len(none_terminal_next_states) != 0:
    Q_s_prime_a_prime[none_terminal_next_state_index] = my_agent.target_model(none_terminal_next_states).detach().max(1)[0].unsqueeze(1)
Q_s_prime_a_prime.size()

torch.Size([32, 1])

In [118]:
target = reward + 0.99 * Q_s_prime_a_prime
target

tensor([[0.0429],
        [0.0429],
        [0.0429],
        [0.0429],
        [0.0429],
        [0.0429],
        [0.0429],
        [0.0429],
        [0.0429],
        [0.0429],
        [0.0429],
        [0.0429],
        [0.0429],
        [0.0429],
        [0.0429],
        [0.0429],
        [0.0429],
        [0.0429],
        [0.0429],
        [0.0429],
        [0.0429],
        [0.0429],
        [0.0429],
        [0.0429],
        [0.0429],
        [0.0429],
        [0.0429],
        [0.0429],
        [0.0429],
        [0.0429],
        [0.0429],
        [0.0429]])

In [120]:
import torch.nn.functional as f

loss = f.smooth_l1_loss(target, Q_s_a)


In [121]:
loss

tensor(0.0024, grad_fn=<SmoothL1LossBackward0>)